<a href="https://colab.research.google.com/github/jcms2665/TallerR_UDLAP/blob/master/COVID_19_en_CDMX_JC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Librerías { display-mode: "form" }
%matplotlib inline
!pip install arviz
import numpy as np
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import seaborn as sns
import arviz as az
import pymc3 as pm
import altair as alt
import plotly.express as px
import requests
import io

sns.set_context('talk')
plt.style.use('seaborn-whitegrid')

<h1><center> Desarrollo de COVID - 19 Ciudad de México </center></h1>

#Contexto

Tomado del muro de **Miguel Angel Núñez Ochoa** quien hace el ejercicio para Guadalajara y adaptado con los datos de la Ciudad de México. 

**Cualquier error u observación es importante**

---
#Introducción al modelo SEIR

El modelo con el cuál vamos a trabajar es conocido como **SEIR** y la implementación que hice está basada en el libro: *Modeling Infectious Diseases in humans and animals (Keeling and Rohani, 2008)* 

Este modelo es uno de los modelos epidemiológicos más sencillos, se llama así por sus siglas,que hacen referencia a la capacidad del portador para transmitir la enfermedad:

* **S**uceptible: Persona aún no expuesta al patógeno, pero forma parte de una población que ya tiene expuestos.

* **E**xpuesto: Aquellas personas que están infectados pero aun no son *infecciosos*
 * Estas personas ya tienen el virus, pero el virus se encuentra en una etapa de desarrollo en el cuerpo, la carga viral es tán poca que la transmisión activa del virus es poco probable.
* **I**nfectados: Personas colonizadas por el patógeno
* **R**ecuperados: Aquellos que ya resolvieron la enfermedad.

>>>>![SEIR](https://drive.google.com/uc?id=16GNLZ5a8zZctmzWjMECjKrUZ4Zn4hwwQ)

---
#Modelo matemático
<h2><center>¿Cómo modelamos esto?</center></h2>

Al ser un sistema dinámico, evoluciona con el tiempo, este tipo de sistemas se modelan con ecuaciones diferenciales, en el caso específico del modelo **SEIR** las ecuaciones son las siguientes:

\begin{gather*} 
\dot{S} =  - \beta SI \\ 
\dot{E} =  \beta SI - \alpha E \\
\dot{I} =  - \alpha E - \gamma I \\ 
\dot{R} =  \gamma I \\
S+E+I+R = 1
\end{gather*}

1. La primera ecuación describe el cambio en personas suceptibles
 * $\beta\$ es el nombre que le daremos a la **tasa promedio de contagio**

<font color='red'>FACT:</font> el distanciamiento social y la cuarentena son inversamente proporcinales a $\beta\$, es decir:

<center> "Al aumentar el distanciamento social, disminuimos la tasa promedio de contagio, por que reducimos el número de individuos infectados que se están mezclando de manera libre en la población" </center>

2. La segunda ecuación describe a las personas expuestas:
 * $\alpha$ es el **inverso del periodo de incubación**
 * Según (Laurer et al. 2020; Hellewell et al. 2020) el periodo de incubación para COVID-19 es de alrededor de **5 dias**, por lo tanto:
  * $\alpha = 1 / 5 \rightarrow 0.2$

3. La tercera ecuación modela el cambio en personas infectadas, basada en la gente expuesta (aquellas infectadas pero que aun no son *infecciosas*) y el tiempo promedio de capacidad infecciosa, es decir, cuantas personas expuestas hay y cuanto tiempo cargan la capacidad de infectar.
 * $\gamma$ es el inverso del **tiempo promedio de capacidad infecciosa**

4. La cuarta ecuación modela a los recuperados.


---

<h1><center>CIUDAD DE MÉXICO</center></h1>

**Supuestos para la ciudad de México:**

* Población ($N$) de $\sim$ 8510800 habitantes (INEGI).
* 367 infectados al 23/03/2020 ($I$)
* Las 367 personas han expuesto a 4 más (familias de 4 personas)
* Evolución de la enfermedad por 90 días.

In [0]:
dias_evaluacion = 90 # 90 días
dt = 1
periodo_evaluacion = np.linspace(0, dias_evaluacion, dias_evaluacion + 1)

alpha = 0.2
beta = 1.75
gamma = 0.5
parametros = alpha, beta, gamma

#Condiciones iniciales de la ZMG
ZMG_Population = 8510800 
I_o = 367 / ZMG_Population  # 367 casos al 24-03-2020
E_o = (367*4)/ZMG_Population # 4 Expuestos por caso
S_o = (1) - (E_o+I_o) # El resto somos suceptibles
R_o = 1,865 # Recuperado

Condiciones_Iniciales = S_o,E_o,I_o,R_o

In [0]:
def SEIR(IC,pmtrs,t,theta=1):
 
  So,Eo,Io,Ro = IC
  S,E,I,R = [So],[Eo],[Io],[Ro]
  alpha,beta,gamma = pmtrs
  dt = t[1]-t[0]
  for _ in t[1:]:
    St = S[-1] - (theta*beta*S[-1]*I[-1])*dt
    Et = E[-1] + (theta*beta*S[-1]*I[-1] - alpha*E[-1])*dt
    It = I[-1] + (alpha*E[-1] - gamma*I[-1])*dt
    Rt = R[-1] + (gamma*I[-1])*dt
    S.append(St)
    E.append(Et)
    I.append(It)
    R.append(Rt)
  return S,E,I,R

In [0]:
S,E,I,R = SEIR(Condiciones_Iniciales,parametros,periodo_evaluacion) # Obtenemos los suceptibles, expuestos, infectados y recuperados para los proximos 90 dias

In [0]:
#@title Ciudad de México { display-mode: "form" }

E_a = np.array(E)*ZMG_Population
I_a = np.array(I)*ZMG_Population
Clase = np.array(["Expuestos"]*np.array(E).shape[0] + ["Infectados"]*np.array(E).shape[0])
Dias = np.concatenate([np.arange(0,91,1),np.arange(0,91,1)])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([E_a,I_a]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase')

fig.update_layout(
    title="Predicción del modelo SEIR de la evolución de COVID-19 en Ciudad de México",
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )
for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()